# <center> BIT522 – Machine Learning (Spring 2018)</center>
# <center>Homework #4<center>

# <center> Harlinton Palacios Mosquera</center>
# <center>161041033</center>

In [1]:
import numpy as np
from sklearn import manifold, datasets, decomposition, discriminant_analysis
from keras.datasets import mnist
from keras import backend as K
import keras
from scipy.spatial.distance import pdist, squareform
import numpy as np

C:\Users\Harlinton\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# <center>Data</center>

In [2]:
#X, y = make_moons(n_samples=100, random_state=123)
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 784) 
x_test = x_test.reshape(10000, 784)
indices = np.random.permutation(np.arange(x_train.shape[0]))
x_train = x_train[indices]
y_train = y_train[indices]
y_train
x_train.shape

(60000, 784)

#  <center>Part I: Implementing PCA</center>


In [3]:
from time import time
from sklearn.decomposition import PCA
n_components = 100
t0 = time()
pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(x_train)
print("done in %0.3fs" % (time() - t0))

X_train_pca = pca.transform(x_train)
X_test_pca = pca.transform(x_test)
x_train.shape

done in 7.864s


(60000, 784)

# <center>Part II: Using PCA before Classification (k-means algorithm)

# 1)  Euclidean Distance

In [4]:
def cluster_centroids(data, clusters, k=None):

    if k is None:
        k = np.max(clusters) + 1
    result = np.empty(shape=(k,) + data.shape[1:])
    for i in range(k):
        np.mean(data[clusters == i], axis=0, out=result[i])
    return result

In [5]:
import scipy.spatial

def kmeans(data, k=None, centroids=None, steps=20):
 
    if centroids is not None and k is not None:
        assert(k == len(centroids))
    elif centroids is not None:
        k = len(centroids)
    elif k is not None:
        # Forgy initialization method: choose k data points randomly.
        centroids = data[np.random.choice(np.arange(len(data)), k, False)]
    else:
        raise RuntimeError("Need a value for k or centroids.")

    for _ in range(max(steps, 1)):
        
        # Euclidean  distance between the points
        
        sqdists = scipy.spatial.distance.cdist(centroids, data, 'sqeuclidean')
        
        # Manhattan distance between the points
        #sqdists = scipy.spatial.distance.cdist(centroids, data, 'cityblock')
                
        #Cosine distance between the points
        #qdists = scipy.spatial.distance.cdist(centroids, data, 'cosine')


         
        # Index of the closest centroid to each data point.
        clusters = np.argmin(sqdists, axis=0)

        new_centroids = cluster_centroids(data, clusters, k)
        if np.array_equal(new_centroids, centroids):
            break

        centroids = new_centroids
    #print (centroids)
    
    return clusters , centroids

In [6]:
k= 10
# cluster_label, centroids = kmeans(x_train,k)

#cross validation
from sklearn.model_selection import KFold
k_fold= 5
kf = KFold(n_splits = k_fold) # Define the split - into 5 folds 
kf.get_n_splits(X_train_pca) # returns the number of splitting iterations in the cross-validator

centroids = None
for train_index, test_index in kf.split(X_train_pca):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train_cv, X_test_cv = X_train_pca[train_index], X_train_pca[test_index]
    cluster_label, centroids = kmeans(X_train_cv, k, centroids=centroids)

cluster_label, centroids = kmeans(X_train_pca, k, centroids=centroids)


In [7]:
table = np.zeros((10, 10),dtype=np.int)

for i, k  in enumerate(y_train):
    c = cluster_label[i]
    table[k,c] += 1        

In [8]:
print (table)

[[ 469    6    5  111 2511 2265  161   12  148  235]
 [ 519   14   36   44   47    0   18 6018   19   27]
 [ 429    6   88 1381 1073   54   53  201 1701  972]
 [ 244  262  101  252 4310    3   62  126  105  666]
 [1001  393 2351   12  365   13  608  302  179  618]
 [1352  213   84   75 1294   25 1648  107  126  497]
 [ 313    0    8 2187  106  342    9   72 2851   30]
 [   7 2806 1644    0  146   32  218 1287   44   81]
 [ 646  202  227   21 2935   44  789  393   57  537]
 [  15 2023 2449    0  214   23   52  643   22  508]]


In [9]:
dic = {}
while len(dic) != 10:
    index = table.argmax()
    x = index // 10
    y = index % 10
    if y not in dic.keys() and x not in dic.values():
        #print (table[x, y])
        dic[y] = x
    table[x, y] = -1
#print(dic)

In [10]:
y_predict = []
for y in cluster_label:
    y_predict.append(dic[y])
y_predict = np.array(y_predict)

In [11]:
#cluster_label

In [12]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
confuMatrix= confusion_matrix(y_train, y_predict)
accuracy = accuracy_score(y_train, y_predict)*100

print ("Confusion Matrix: \n\n" , confuMatrix)
print ("\nAccuracy: " , accuracy)

Confusion Matrix: 

 [[2265   12  111 2511  469  161  148    6  235    5]
 [   0 6018   44   47  519   18   19   14   27   36]
 [  54  201 1381 1073  429   53 1701    6  972   88]
 [   3  126  252 4310  244   62  105  262  666  101]
 [  13  302   12  365 1001  608  179  393  618 2351]
 [  25  107   75 1294 1352 1648  126  213  497   84]
 [ 342   72 2187  106  313    9 2851    0   30    8]
 [  32 1287    0  146    7  218   44 2806   81 1644]
 [  44  393   21 2935  646  789   57  202  537  227]
 [  23  643    0  214   15   52   22 2023  508 2449]]

Accuracy:  42.11


In [13]:
sqdists = scipy.spatial.distance.cdist(centroids, X_test_pca, 'sqeuclidean')
y_test_predict = sqdists.argmin(axis=0)

y_test_predict1 = []
for y in y_test_predict:
    y_test_predict1.append(dic[y])
y_test_predict1

ConfuMatrixnn = confusion_matrix(y_test, y_test_predict1)
accuracynn =accuracy_score(y_test, y_test_predict1)*100

print ("Confusion Matrix: \n\n" , ConfuMatrixnn)
print ("\nAccuracy: " , accuracynn)

Confusion Matrix: 

 [[ 370    1   20  453   59   14   28    3   32    0]
 [   0 1024    5   10   84    1    6    0    2    3]
 [  10   53  244  232   53    7  288    4  137    4]
 [   1   20   33  744   30    5   18   45   99   15]
 [   3   59    1   52  171   91   34   54   89  428]
 [   4   18    8  208  241  260   15   45   76   17]
 [  76    8  406   21   51    1  388    0    5    2]
 [   0  205    0   25    1   37   13  460   16  271]
 [   4   44    8  514  106  112    8   51   85   42]
 [   3   77    0   41    3    8    3  324   69  481]]

Accuracy:  42.27


# <center>Results Of The Other Distance</center>

## 2) L1 norm (Manhattan distance)


<b>Labeling of clusters:</b> 

<img src="https://image.ibb.co/cDRRYJ/Table1.png" alt="Drawing" style="width: 500px;"/>

<b>Training error :</b> 
<img src="https://image.ibb.co/nqqmYJ/resul1.png" alt="Drawing" style="width: 500px;"/>

<b>Test error with 1-NN:</b> 
<img src="https://image.ibb.co/dyCdfy/MatriNN.png" alt="Drawing" style="width: 500px;"/>

## 3) Cosine distance.

<b>Labeling of clusters:</b> 

<img src="https://image.ibb.co/m6ybHd/Cosine_Table.png" alt="Drawing" style="width: 500px;"/>

<b>Training error :</b> 
<img src="https://image.ibb.co/kBAZ3J/matris1cosine.png" alt="Drawing" style="width: 500px;"/>

<b>Test error with 1-NN:</b> 
<img src="https://image.ibb.co/eeC3xd/Matrixconcine_NN.png" alt="Drawing" style="width: 500px;"/>

K means is a heuristic algorithm that partitions a data set into 
K clusters by minimizing the sum of squared distance in each 
cluster.  During  the  implementation  of  k
-
means  with  three 
different  d
istance  metrics,  it  is  observed  that  selection  of 
distance  metric  plays  a  very  important  role  in  clustering.  So, 
the selection of distance metric should be made carefully. The 
distortion  in  k
-
means  using  Cosine distance  metric  is  less 
than that of k
-
mea
ns using Euclidean and Manhattan distance metric.
As  a  conclusion,  the  K
-
means,  which  is  implemented  using 
Euclidean  distance  metric  gives  best  result  and  K
-
means 
based on Cosine distance metric’s performance, is worst.